<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/HybridEfficientNetHYbrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== Cell 1: Mount Drive & locate EfficientNetV2B0 =====
import os, fnmatch, datetime
from google.colab import drive
import shutil

# You usually don't need to clear the mountpoint, but keeping your logic:
mountpoint = '/content/drive'
if os.path.exists(mountpoint):
    print(f"Clearing existing mountpoint: {mountpoint}")
    try:
        for item in os.listdir(mountpoint):
            item_path = os.path.join(mountpoint, item)
            if os.path.isdir(item_path):
                shutil.rmtree(item_path)
            else:
                os.remove(item_path)
    except Exception as e:
        print(f"Error clearing mountpoint: {e}")

drive.mount(mountpoint, force_remount=True)

BASE   = f"{mountpoint}/MyDrive/DiabeticProject"
SAVED  = f"{BASE}/saved_models"
TFLITE = f"{BASE}/tflite"
HYBRID_SAVE_DIR = f"{BASE}/hybrid_models"
os.makedirs(HYBRID_SAVE_DIR, exist_ok=True)

def find_one(root, patterns=(".keras", ".h5"), name_contains=("efficientnetv2b0", "efficientnet", "v2b0")):
    hits = []
    for r, _, files in os.walk(root):
        for f in files:
            if not f.lower().endswith(patterns):
                continue
            if any(k in f.lower() for k in name_contains):
                hits.append(os.path.join(r, f))
    hits.sort(key=lambda p:(-os.path.getmtime(p), len(p)))
    return hits[0] if hits else None

def find_tflite(root, name_contains=("efficientnetv2b0", "efficientnet", "v2b0"), quant="fp16"):
    hits = []
    for r, _, files in os.walk(root):
        for f in files:
            if not f.lower().endswith(".tflite"):
                continue
            if any(k in f.lower() for k in name_contains) and quant in f.lower():
                hits.append(os.path.join(r, f))
    hits.sort(key=lambda p:(-os.path.getmtime(p), len(p)))
    return hits[0] if hits else None

# Pick EfficientNet files (from your listing)
EFFICIENTNET_KERAS_PATH  = find_one(SAVED, (".keras", ".h5"), ("efficientnetv2b0", "efficientnet", "v2b0"))
EFFICIENTNET_TFLITE_FP16 = find_tflite(TFLITE, ("efficientnetv2b0", "efficientnet", "v2b0"), "fp16")

# Fallback to your exact known file names if discovery missed:
if not EFFICIENTNET_KERAS_PATH:
    cand = f"{SAVED}/EfficientNetV2B0_single_split.keras"
    if os.path.exists(cand): EFFICIENTNET_KERAS_PATH = cand
if not EFFICIENTNET_TFLITE_FP16:
    cand = f"{TFLITE}/EfficientNetV2B0_model_fp16.tflite"
    if os.path.exists(cand): EFFICIENTNET_TFLITE_FP16 = cand

print("=== EfficientNetV2B0 paths ===")
print("EFFICIENTNET_KERAS_PATH  :", EFFICIENTNET_KERAS_PATH,  "exists:", os.path.exists(EFFICIENTNET_KERAS_PATH) if EFFICIENTNET_KERAS_PATH else None)
print("EFFICIENTNET_TFLITE_FP16 :", EFFICIENTNET_TFLITE_FP16, "exists:", os.path.exists(EFFICIENTNET_TFLITE_FP16) if EFFICIENTNET_TFLITE_FP16 else None)
print("HYBRID_SAVE_DIR          :", HYBRID_SAVE_DIR)

if not EFFICIENTNET_KERAS_PATH:
    raise SystemExit("❌ EfficientNetV2B0 Keras model not found under saved_models.")


Mounted at /content/drive
=== EfficientNetV2B0 paths ===
EFFICIENTNET_KERAS_PATH  : /content/drive/MyDrive/DiabeticProject/saved_models/EfficientNetV2B0_single_split.keras exists: True
EFFICIENTNET_TFLITE_FP16 : /content/drive/MyDrive/DiabeticProject/tflite/EfficientNetV2B0_model_fp16.tflite exists: True
HYBRID_SAVE_DIR          : /content/drive/MyDrive/DiabeticProject/hybrid_models


In [ ]:
# ===== Cell 2: Locate APTOS Binary via KaggleHub & build dataframe =====
import os, re, zipfile, pandas as pd
from sklearn.model_selection import train_test_split

try:
    _ = subhajeetdas_aptos_2019_jpg_path
except NameError:
    import kagglehub
    subhajeetdas_aptos_2019_jpg_path = kagglehub.dataset_download('subhajeetdas/aptos-2019-jpg')

print("KaggleHub root:", subhajeetdas_aptos_2019_jpg_path)

def _norm(s): return re.sub(r'[_\s]+', ' ', s.strip().lower())
def _has_binary(dirpath):
    try:
        kids = [_norm(d) for d in os.listdir(dirpath) if os.path.isdir(os.path.join(dirpath, d))]
        return ('dr' in kids) and ('no dr' in kids)
    except Exception:
        return False

def _maybe_unzip_here(root):
    for f in os.listdir(root):
        if f.lower().endswith(".zip") and "aptos" in f.lower() and "binary" in f.lower():
            z = os.path.join(root, f)
            out = os.path.join(root, os.path.splitext(f)[0])
            if not os.path.isdir(out):
                print("📦 Extracting:", z)
                with zipfile.ZipFile(z, 'r') as zf: zf.extractall(out)

def find_binary_root():
    _maybe_unzip_here(subhajeetdas_aptos_2019_jpg_path)
    cands = [
        os.path.join(subhajeetdas_aptos_2019_jpg_path, "APTOS 2019 (Original) (Binary)"),
        subhajeetdas_aptos_2019_jpg_path,
    ]
    for c in cands:
        if os.path.isdir(c) and _has_binary(c): return c
    for dirpath, _, files in os.walk(subhajeetdas_aptos_2019_jpg_path):
        if _has_binary(dirpath): return dirpath
        if any(f.lower().endswith(".zip") for f in files): _maybe_unzip_here(dirpath)
    for c in [
        "/content/drive/MyDrive/DiabeticProject/APTOS 2019 (Original) (Binary)",
        "/content/drive/MyDrive/APTOS 2019 (Original) (Binary)"
    ]:
        if os.path.isdir(c) and _has_binary(c): return c
    raise SystemExit("❌ Could not locate the APTOS Binary dataset.")

base_path = find_binary_root()
print("✅ Using dataset root:", base_path)
print("Class folders:", os.listdir(base_path))

exts = ('.png','.jpg','.jpeg','.bmp','.tif','.tiff','.webp')
rows = []
for cls in os.listdir(base_path):
    d = os.path.join(base_path, cls)
    if not os.path.isdir(d): continue
    lab = 1 if _norm(cls)=='dr' else 0 if _norm(cls)=='no dr' else None
    if lab is None: continue
    for f in os.listdir(d):
        if f.lower().endswith(exts):
            rows.append((os.path.join(d, f), lab))
df = pd.DataFrame(rows, columns=['image_path','label'])

train_df, temp_df = train_test_split(df, test_size=0.30, stratify=df['label'], random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.50, stratify=temp_df['label'], random_state=42)
for name, d in [("Train",train_df),("Valid",valid_df),("Test",test_df)]:
    print(f"{name}: total={len(d)} | No DR={(d['label']==0).sum()} | DR={(d['label']==1).sum()}")


100%|██████████| 2.82G/2.82G [00:33<00:00, 89.8MB/s]

Extracting files...


KaggleHub root: /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12
✅ Using dataset root: /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)
Class folders: ['Details.txt', 'No DR', 'DR']
Train: total=2563 | No DR=1263 | DR=1300
Valid: total=549 | No DR=271 | DR=278
Test: total=550 | No DR=271 | DR=279


In [ ]:
# ===== Cell 3: Deduplicate + grouped split (no leakage) =====
import os, re, hashlib, numpy as np, pandas as pd
from PIL import Image
from sklearn.model_selection import GroupShuffleSplit

pool = pd.concat([train_df, valid_df, test_df], ignore_index=True).drop_duplicates("image_path")
pool["label"] = pool["label"].astype(int)

def sha1(fp, chunk=1<<20):
    h = hashlib.sha1()
    with open(fp, 'rb') as f:
        while True:
            b = f.read(chunk)
            if not b: break
            h.update(b)
    return h.hexdigest()

def ahash(path, n=8):
    try:
        img = Image.open(path).convert('L').resize((n,n))
    except Exception:
        return None
    arr = np.asarray(img, dtype=np.float32)
    bits = (arr > arr.mean()).astype(np.uint8).ravel()
    out = []
    for i in range(0, bits.size, 8):
        byte = 0
        for j in range(8):
            if i+j < bits.size and bits[i+j]: byte |= (1 << (7-j))
        out.append(byte)
    return tuple(out)

print("🔍 SHA1 …")
pool["sha1"] = pool["image_path"].apply(sha1)
pool = pool.drop_duplicates("sha1").reset_index(drop=True)

print("🔍 aHash …")
pool["ahash"] = pool["image_path"].apply(ahash)
pool = pool[pool["ahash"].notna()].drop_duplicates("ahash").reset_index(drop=True)

def pid_from_path(p):
    stem = os.path.splitext(os.path.basename(p))[0]
    return re.sub(r'_(left|right)$', '', stem, flags=re.IGNORECASE)

pool["group"] = pool["image_path"].apply(pid_from_path)

gss = GroupShuffleSplit(n_splits=1, train_size=0.70, random_state=42)
i_tr, i_tmp = next(gss.split(pool, groups=pool["group"]))
df_tr, df_tmp = pool.iloc[i_tr].reset_index(drop=True), pool.iloc[i_tmp].reset_index(drop=True)

gss2 = GroupShuffleSplit(n_splits=1, train_size=0.50, random_state=42)
i_va, i_te = next(gss2.split(df_tmp, groups=df_tmp["group"]))
df_va, df_te = df_tmp.iloc[i_va].reset_index(drop=True), df_tmp.iloc[i_te].reset_index(drop=True)

def _cnt(tag, d):
    print(f"{tag}: total={len(d)} | No DR={(d['label']==0).sum()} | DR={(d['label']==1).sum()} | groups={d['group'].nunique()}")
print("\n✅ Post-dedup grouped split:")
_cnt("Train", df_tr); _cnt("Valid", df_va); _cnt("Test ", df_te)
print("Group overlaps:",
      len(set(df_tr.group)&set(df_va.group)),
      len(set(df_tr.group)&set(df_te.group)),
      len(set(df_va.group)&set(df_te.group)))

train_df, valid_df, test_df = df_tr.drop(columns=["sha1","ahash"]), df_va.drop(columns=["sha1","ahash"]), df_te.drop(columns=["sha1","ahash"])


🔍 SHA1 …
🔍 aHash …

✅ Post-dedup grouped split:
Train: total=1242 | No DR=528 | DR=714 | groups=1213
Valid: total=265 | No DR=121 | DR=144 | groups=260
Test : total=265 | No DR=116 | DR=149 | groups=260
Group overlaps: 0 0 0


In [ ]:
# ===== Cell 3: Deduplicate + grouped split (no leakage) — works for EfficientNetV2B0 too =====
import os, re, hashlib, numpy as np, pandas as pd
from PIL import Image
from sklearn.model_selection import GroupShuffleSplit

pool = pd.concat([train_df, valid_df, test_df], ignore_index=True).drop_duplicates("image_path")
pool["label"] = pool["label"].astype(int)

def sha1(fp, chunk=1<<20):
    h = hashlib.sha1()
    with open(fp, 'rb') as f:
        while True:
            b = f.read(chunk)
            if not b: break
            h.update(b)
    return h.hexdigest()

def ahash(path, n=8):
    try:
        img = Image.open(path).convert('L').resize((n,n))
    except Exception:
        return None
    arr = np.asarray(img, dtype=np.float32)
    bits = (arr > arr.mean()).astype(np.uint8).ravel()
    out = []
    for i in range(0, bits.size, 8):
        byte = 0
        for j in range(8):
            if i+j < bits.size and bits[i+j]: byte |= (1 << (7-j))
        out.append(byte)
    return tuple(out)

print("🔍 SHA1 …")
pool["sha1"] = pool["image_path"].apply(sha1)
pool = pool.drop_duplicates("sha1").reset_index(drop=True)

print("🔍 aHash …")
pool["ahash"] = pool["image_path"].apply(ahash)
pool = pool[pool["ahash"].notna()].drop_duplicates("ahash").reset_index(drop=True)

def pid_from_path(p):
    stem = os.path.splitext(os.path.basename(p))[0]
    return re.sub(r'_(left|right)$', '', stem, flags=re.IGNORECASE)

pool["group"] = pool["image_path"].apply(pid_from_path)

gss = GroupShuffleSplit(n_splits=1, train_size=0.70, random_state=42)
i_tr, i_tmp = next(gss.split(pool, groups=pool["group"]))
df_tr, df_tmp = pool.iloc[i_tr].reset_index(drop=True), pool.iloc[i_tmp].reset_index(drop=True)

gss2 = GroupShuffleSplit(n_splits=1, train_size=0.50, random_state=42)
i_va, i_te = next(gss2.split(df_tmp, groups=df_tmp["group"]))
df_va, df_te = df_tmp.iloc[i_va].reset_index(drop=True), df_tmp.iloc[i_te].reset_index(drop=True)

def _cnt(tag, d):
    print(f"{tag}: total={len(d)} | No DR={(d['label']==0).sum()} | DR={(d['label']==1).sum()} | groups={d['group'].nunique()}")
print("\n✅ Post-dedup grouped split:")
_cnt("Train", df_tr); _cnt("Valid", df_va); _cnt("Test ", df_te)
print("Group overlaps:",
      len(set(df_tr.group)&set(df_va.group)),
      len(set(df_tr.group)&set(df_te.group)),
      len(set(df_va.group)&set(df_te.group)))

train_df, valid_df, test_df = df_tr.drop(columns=["sha1","ahash"]), df_va.drop(columns=["sha1","ahash"]), df_te.drop(columns=["sha1","ahash"])


🔍 SHA1 …
🔍 aHash …

✅ Post-dedup grouped split:
Train: total=1242 | No DR=528 | DR=714 | groups=1213
Valid: total=265 | No DR=121 | DR=144 | groups=260
Test : total=265 | No DR=116 | DR=149 | groups=260
Group overlaps: 0 0 0


In [ ]:
# ===== Cell 4 (TFLite-FIRST, EfficientNetV2B0): extract features by tapping pooled tensor (1280-d) =====
import os, numpy as np, tensorflow as tf
from PIL import Image

# EXPECTED GLOBALS: EFFICIENTNET_TFLITE_FP16, train_df, valid_df, test_df
assert EFFICIENTNET_TFLITE_FP16 and os.path.exists(EFFICIENTNET_TFLITE_FP16), \
    f"Full fp16 TFLite not found: {EFFICIENTNET_TFLITE_FP16}"

def _load_tflite(path):
    inter = tf.lite.Interpreter(model_path=path, num_threads=os.cpu_count() or 1)
    inter.allocate_tensors()
    return inter

def _prep(img_path, W, H, expect_dtype):
    img = Image.open(img_path).convert("RGB").resize((W, H), Image.BILINEAR)
    x = np.asarray(img, dtype=np.float32) / 255.0
    x = np.expand_dims(x, 0)
    if expect_dtype == np.float16: x = x.astype(np.float16)
    elif expect_dtype == np.float32: x = x.astype(np.float32)
    else: x = x.astype(expect_dtype)
    return x

def _find_feature_index(inter, preferred_dims=(1280,)):
    details = inter.get_tensor_details()
    cand = None
    # name-based hints first
    for d in details:
        name = d["name"].decode() if isinstance(d["name"], bytes) else d["name"]
        shp = tuple(int(s) for s in (d.get("shape_signature", d["shape"])))
        if len(shp) in (1,2) and any(k in name.lower() for k in
                                     ("global_average_pool","global_max_pool","avg_pool","mean","gap","pool")):
            cand = d
    # dim-based fallback (EfficientNetV2B0 penultimate = 1280)
    if cand is None:
        for d in details:
            shp = tuple(int(s) for s in (d.get("shape_signature", d["shape"])))
            if len(shp) in (1,2) and shp[-1] in preferred_dims:
                cand = d
                break
    if cand is not None:
        print(f"🔎 Using feature tensor idx={cand['index']} name={cand.get('name')} shape={cand.get('shape')}")
        return cand["index"]
    print("⚠️ No internal pool tensor matched — falling back to model output.")
    return inter.get_output_details()[0]["index"]

def extract_features_from_full_tflite(tflite_path, df):
    inter = _load_tflite(tflite_path)
    inp = inter.get_input_details()[0]
    H, W, C = int(inp["shape"][1]), int(inp["shape"][2]), int(inp["shape"][3])
    feat_idx = _find_feature_index(inter, preferred_dims=(1280,))
    X, y = [], df["label"].astype(int).to_numpy()
    for p in df["image_path"]:
        a = _prep(p, W, H, inp["dtype"])
        inter.set_tensor(inp["index"], a)
        inter.invoke()
        f = inter.get_tensor(feat_idx).reshape(-1)
        X.append(f)
    X = np.vstack(X)
    print("📐 Feature matrix:", X.shape, "from", os.path.basename(tflite_path))
    return X, y

print("✅ Using FULL TFLite for feature extraction:", EFFICIENTNET_TFLITE_FP16)
Xe_tr, ye_tr = extract_features_from_full_tflite(EFFICIENTNET_TFLITE_FP16, train_df)
Xe_va, ye_va = extract_features_from_full_tflite(EFFICIENTNET_TFLITE_FP16, valid_df)
Xe_te, ye_te = extract_features_from_full_tflite(EFFICIENTNET_TFLITE_FP16, test_df)


✅ Using FULL TFLite for feature extraction: /content/drive/MyDrive/DiabeticProject/tflite/EfficientNetV2B0_model_fp16.tflite
🔎 Using feature tensor idx=739 name=functional_1_1/efficientnetv2-b0_1/max_pool_1/Max shape=[   1 1280]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


📐 Feature matrix: (1242, 1280) from EfficientNetV2B0_model_fp16.tflite
🔎 Using feature tensor idx=739 name=functional_1_1/efficientnetv2-b0_1/max_pool_1/Max shape=[   1 1280]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


📐 Feature matrix: (265, 1280) from EfficientNetV2B0_model_fp16.tflite
🔎 Using feature tensor idx=739 name=functional_1_1/efficientnetv2-b0_1/max_pool_1/Max shape=[   1 1280]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


📐 Feature matrix: (265, 1280) from EfficientNetV2B0_model_fp16.tflite


In [ ]:
# ===== Cell 5B (dimension-safe): recover/train heads → auto-fix dim mismatch → distill/export head TFLite → time =====
import os, glob, time, json, joblib, numpy as np, pandas as pd
import tensorflow as tf
from PIL import Image
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# ---------- 0) Resolve backbone + features (prefer EfficientNet) ----------
if 'EFFICIENTNET_TFLITE_FP16' in globals() and EFFICIENTNET_TFLITE_FP16 and os.path.exists(EFFICIENTNET_TFLITE_FP16):
    BACKBONE_TFLITE = EFFICIENTNET_TFLITE_FP16
    TAG = "EfficientNetV2B0_fp16_FE"
    X_tr = globals().get('Xe_tr'); X_va = globals().get('Xe_va')
    y_tr = globals().get('ye_tr'); y_va = globals().get('ye_va')
    expected_feat_dim = 1280  # EfficientNetV2B0 pooled width
else:
    assert 'DENSENET_TFLITE_FP16' in globals() and os.path.exists(DENSENET_TFLITE_FP16), \
        "No EfficientNet or DenseNet backbone TFLite found."
    BACKBONE_TFLITE = DENSENET_TFLITE_FP16
    TAG = "DenseNet121_fp16_FE"
    X_tr = globals().get('Xd_tr'); X_va = globals().get('Xd_va')
    y_tr = globals().get('yd_tr'); y_va = globals().get('yd_va')
    expected_feat_dim = 1024  # DenseNet121 pooled width

assert isinstance(X_tr, np.ndarray) and isinstance(X_va, np.ndarray), "Feature matrices (X_tr/X_va) not found."
assert isinstance(y_tr, np.ndarray) and isinstance(y_va, np.ndarray), "Label vectors (y_tr/y_va) not found."
assert 'test_df' in globals() and len(test_df) > 0 and 'image_path' in test_df.columns, "test_df with image_path required."

SAVE_DIR = HYBRID_SAVE_DIR if 'HYBRID_SAVE_DIR' in globals() else "./hybrid_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# ---------- 1) Recover/Train heads ----------
heads = {}
# (a) in-RAM
if 'heads' in globals() and isinstance(heads, dict) and heads:
    pass
else:
    if 'knn' in globals(): heads['KNN'] = knn
    if 'svm' in globals(): heads['SVM_RBF'] = svm
    if 'rf'  in globals(): heads['RandomForest'] = rf

# (b) from disk
def _try_load_head(path):
    try:
        return joblib.load(path)
    except Exception:
        return None

if not heads:
    cand_paths = [
        os.path.join(SAVE_DIR, f"{TAG}_KNN.joblib"),
        os.path.join(SAVE_DIR, f"{TAG}_SVM_RBF.joblib"),
        os.path.join(SAVE_DIR, f"{TAG}_RandomForest.joblib"),
    ]
    cand_paths += glob.glob(os.path.join(SAVE_DIR, "*_KNN.joblib"))
    cand_paths += glob.glob(os.path.join(SAVE_DIR, "*_SVM_RBF.joblib"))
    cand_paths += glob.glob(os.path.join(SAVE_DIR, "*_RandomForest.joblib"))
    for p in cand_paths:
        est = _try_load_head(p)
        if est is None: continue
        if p.endswith("KNN.joblib"): heads.setdefault("KNN", est)
        elif p.endswith("SVM_RBF.joblib"): heads.setdefault("SVM_RBF", est)
        elif p.endswith("RandomForest.joblib"): heads.setdefault("RandomForest", est)

# (c) train if still missing
d_in = X_tr.shape[1]
pca_n_default = 128 if d_in > 128 else None

def _mk_steps(use_pca=True, pca_n=pca_n_default):
    steps = [("scaler", StandardScaler())]
    if use_pca and pca_n:
        steps.append(("pca", PCA(n_components=pca_n, random_state=42)))
    return steps

X_train = np.vstack([X_tr, X_va])
y_train = np.concatenate([y_tr, y_va])

if not heads:
    print("ℹ️ No heads found in RAM/disk — training KNN / SVM_RBF / RF on current features …")
    knn = Pipeline(_mk_steps(True) + [("knn", KNeighborsClassifier(n_neighbors=5, weights="distance", n_jobs=-1))]).fit(X_train, y_train)
    svm = Pipeline(_mk_steps(True) + [("svm", SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=42))]).fit(X_train, y_train)
    rf  = Pipeline(_mk_steps(True) + [("rf", RandomForestClassifier(
            n_estimators=600, max_depth=None, min_samples_leaf=1,
            class_weight="balanced_subsample", n_jobs=-1, random_state=42
         ))]).fit(X_train, y_train)
    heads = {"KNN": knn, "SVM_RBF": svm, "RandomForest": rf}
    joblib.dump(knn, os.path.join(SAVE_DIR, f"{TAG}_KNN.joblib"))
    joblib.dump(svm, os.path.join(SAVE_DIR, f"{TAG}_SVM_RBF.joblib"))
    joblib.dump(rf,  os.path.join(SAVE_DIR, f"{TAG}_RandomForest.joblib"))
    print("✅ Heads trained & saved to:", SAVE_DIR)

# ---------- 2) Auto-fix any feature-dimension mismatches (refit on current features) ----------
def _needs_refit(pipe, x_width):
    # check scaler mean/scale width or PCA components width
    scaler = pipe.named_steps.get("scaler")
    pca    = pipe.named_steps.get("pca")
    if scaler is not None:
        mean = getattr(scaler, "mean_", None)
        if mean is not None and mean.shape[0] != x_width:
            return True
    if pca is not None:
        comps = getattr(pca, "components_", None)
        if comps is not None and comps.shape[1] != x_width:
            return True
    # also check n_features_in_ if present
    nfi = getattr(pipe, "n_features_in_", None)
    if nfi is not None and nfi != x_width:
        return True
    return False

def _refit_like(pipe, X, y):
    # reconstruct a similar pipeline (scaler + (optional PCA) + last estimator with same params)
    steps = []
    if "scaler" in pipe.named_steps:
        steps.append(("scaler", StandardScaler()))
    if "pca" in pipe.named_steps:
        # keep same n_components if valid else clamp
        old_pca = pipe.named_steps["pca"]
        n_comp = getattr(old_pca, "n_components", pca_n_default)
        if isinstance(n_comp, int):
            n_comp = min(n_comp, X.shape[1])
        steps.append(("pca", PCA(n_components=n_comp, random_state=42)))
    # clone last estimator
    last_name, last_est = list(pipe.named_steps.items())[-1]
    est = clone(last_est)
    new_pipe = Pipeline(steps + [(last_name, est)])
    return new_pipe.fit(X, y)

for name in list(heads.keys()):
    if _needs_refit(heads[name], X_train.shape[1]):
        print(f"♻️ Re-fitting {name} to match current feature width {X_train.shape[1]} …")
        heads[name] = _refit_like(heads[name], X_train, y_train)
        # persist
        joblib.dump(heads[name], os.path.join(SAVE_DIR, f"{TAG}_{name}.joblib"))

# ---------- 3) Pick final head (best f1_macro if available, else SVM→KNN→RF) ----------
def _pick_best_head_from_metrics(metrics_df, tag):
    mdf = metrics_df.copy()
    if not {"model","f1_macro"} <= set(mdf.columns): return None
    mdf = mdf[mdf["model"].astype(str).str.startswith(tag + "+")]
    if mdf.empty: return None
    mdf = mdf.assign(head_name=mdf["model"].str.split("+").str[-1])
    return mdf.sort_values("f1_macro", ascending=False)["head_name"].iloc[0]

final_pipe = None
if 'metrics_df' in globals():
    best = _pick_best_head_from_metrics(metrics_df, TAG)
    if best in heads: final_pipe = heads[best]
if final_pipe is None:
    for cand in ("SVM_RBF","KNN","RandomForest"):
        if cand in heads: final_pipe = heads[cand]; break
assert final_pipe is not None, "Could not select a final head."

# ---------- 4) Distill chosen head → LR on its transformed space ----------
def _transform(pipe, X):
    Z = X
    if "scaler" in pipe.named_steps: Z = pipe.named_steps["scaler"].transform(Z)
    if "pca"    in pipe.named_steps: Z = pipe.named_steps["pca"].transform(Z)
    return Z

try:
    prob = final_pipe.predict_proba(X_train)[:, 1]
    y_hard = (np.clip(prob, 1e-6, 1-1e-6) >= 0.5).astype(int)
    Z_train = _transform(final_pipe, X_train)
    lr = LogisticRegression(max_iter=2000, random_state=42).fit(Z_train, y_hard)
except Exception:
    Z_train = _transform(final_pipe, X_train)
    lr = LogisticRegression(max_iter=2000, random_state=42).fit(Z_train, y_train)

# Fuse scaler + PCA + LR back to raw feature basis
scaler = final_pipe.named_steps.get("scaler")
pca    = final_pipe.named_steps.get("pca")
scale = getattr(scaler, "scale_", None) if scaler is not None else None
mean  = getattr(scaler, "mean_",  None) if scaler is not None else None
P     = pca.components_ if pca is not None else None

Wlr = lr.coef_.reshape(-1,1).astype(np.float32)
blr = float(lr.intercept_.ravel()[0])
Wp  = (P.T @ Wlr).astype(np.float32) if P is not None else Wlr
Wf  = (Wp * (1.0 / scale.reshape(-1,1))).astype(np.float32) if scale is not None else Wp
b_shift = - float((Wf.ravel() * mean).sum()) if mean is not None else 0.0
bf = np.array([blr + b_shift], dtype=np.float32)

D_in = Wf.shape[0]
print(f"Head fused: input dim = {D_in}, weights shape = {Wf.shape}, bias = {bf.shape}")

# ---------- 5) Export distilled head as standalone TFLite ----------
head_tfl_path = os.path.join(SAVE_DIR, f"{TAG}_HEAD_fused_fp32.tflite")

class HeadDenseSigmoid(tf.Module):
    def __init__(self, W, b):
        super().__init__()
        self.W = tf.Variable(W, trainable=False, dtype=tf.float32, name="W")
        self.b = tf.Variable(b, trainable=False, dtype=tf.float32, name="b")
    @tf.function(input_signature=[tf.TensorSpec([1, D_in], tf.float32, name="features")])
    def __call__(self, features):
        logits = tf.linalg.matmul(features, self.W) + self.b
        prob = tf.math.sigmoid(logits)
        return {"prob": prob}

_module = HeadDenseSigmoid(Wf, bf)
_concrete = _module.__call__.get_concrete_function()
_converter = tf.lite.TFLiteConverter.from_concrete_functions([_concrete], trackable_obj=_module)
head_tfl_bytes = _converter.convert()
with open(head_tfl_path, "wb") as f:
    f.write(head_tfl_bytes)
print("✅ Wrote HEAD TFLite:", head_tfl_path, "|", round(os.path.getsize(head_tfl_path)/(1024*1024), 3), "MB")

# ---------- 6) Timing helpers ----------
def _load_interp(path):
    inter = tf.lite.Interpreter(model_path=path, num_threads=os.cpu_count() or 1)
    inter.allocate_tensors()
    return inter

def _prep_img(img_path, W, H, dtype):
    img = Image.open(img_path).convert("RGB").resize((W, H), Image.BILINEAR)
    x = np.asarray(img, dtype=np.float32)/255.0
    x = np.expand_dims(x, 0)
    if dtype == np.float16: x = x.astype(np.float16)
    elif dtype == np.float32: x = x.astype(np.float32)
    else: x = x.astype(dtype)
    return x

def _find_feature_index(inter, expected_dim):
    details = inter.get_tensor_details()
    sel = None
    # name-first heuristic
    for d in details:
        name = d["name"].decode() if isinstance(d["name"], bytes) else d["name"]
        shp  = tuple(int(s) for s in (d.get("shape_signature", d["shape"])))
        if len(shp) in (1,2) and any(k in name.lower() for k in ("global_average_pool","global_max_pool","avg_pool","mean","gap","pool")):
            sel = d
    # dim fallback
    if sel is None:
        for d in details:
            shp  = tuple(int(s) for s in (d.get("shape_signature", d["shape"])))
            if len(shp) in (1,2) and shp[-1] == expected_dim:
                sel = d; break
    if sel is None:
        sel = inter.get_output_details()[0]
    return sel["index"]

# ---------- 7) Build 10-image sample & time end-to-end ----------
sample_paths = list(test_df["image_path"])[:10] if len(test_df) >= 10 else list(test_df["image_path"])
assert len(sample_paths) > 0, "No images found in test_df['image_path'] to time."

backbone = _load_interp(BACKBONE_TFLITE)
b_in = backbone.get_input_details()[0]
H, W = int(b_in["shape"][1]), int(b_in["shape"][2])
feat_idx = _find_feature_index(backbone, expected_feat_dim)

head = _load_interp(head_tfl_path)
h_in = head.get_input_details()[0]

# warm-up
for _ in range(3):
    a = _prep_img(sample_paths[0], W, H, b_in["dtype"])
    backbone.set_tensor(b_in["index"], a); backbone.invoke()
    f = backbone.get_tensor(feat_idx).astype(np.float32).reshape(1, D_in)
    head.set_tensor(h_in["index"], f); head.invoke()

times = []
for p in sample_paths:
    a = _prep_img(p, W, H, b_in["dtype"])
    t0 = time.perf_counter()
    backbone.set_tensor(b_in["index"], a); backbone.invoke()
    f = backbone.get_tensor(feat_idx).astype(np.float32).reshape(1, D_in)
    head.set_tensor(h_in["index"], f); head.invoke()
    _ = head.get_output_details()[0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000.0)

t = np.array(times, np.float64)
lat = {"N": len(sample_paths), "avg_ms": round(t.mean(), 3),
       "std_ms": round(t.std(ddof=1), 3) if len(t)>1 else 0.0,
       "min_ms": round(t.min(), 3), "max_ms": round(t.max(), 3)}
df_latency = pd.DataFrame([{**{"BackboneTFLite": BACKBONE_TFLITE, "HeadTFLite": head_tfl_path, "TAG": TAG}, **lat}])
display(df_latency)

lat_csv = os.path.join(SAVE_DIR, f"{TAG}_ALL_TFLite_hybrid_latency.csv")
df_latency.to_csv(lat_csv, index=False)
with open(os.path.join(SAVE_DIR, f"{TAG}_ALL_TFLite_hybrid_manifest.json"), "w") as f:
    json.dump({"backbone_tflite": BACKBONE_TFLITE,
               "head_tflite": head_tfl_path,
               "feature_dim": int(D_in),
               "images_timed": len(sample_paths),
               "latency_ms": lat}, f, indent=2)

print("\n✅ Saved:")
print(" -", head_tfl_path)
print(" -", lat_csv)


♻️ Re-fitting KNN to match current feature width 1280 …
♻️ Re-fitting SVM_RBF to match current feature width 1280 …
♻️ Re-fitting RandomForest to match current feature width 1280 …
Head fused: input dim = 1280, weights shape = (1280, 1), bias = (1,)
✅ Wrote HEAD TFLite: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_fused_fp32.tflite | 0.006 MB


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


,BackboneTFLite,HeadTFLite,TAG,N,avg_ms,std_ms,min_ms,max_ms
0,/content/drive/MyDrive/DiabeticProject/tflite/...,/content/drive/MyDrive/DiabeticProject/hybrid_...,EfficientNetV2B0_fp16_FE,10,32.11,1.464,30.566,35.815



✅ Saved:
 - /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_fused_fp32.tflite
 - /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_ALL_TFLite_hybrid_latency.csv


In [ ]:
# ===== Cell 5C (EfficientNetV2B0): Export THREE head TFLites (from KNN/SVM/RF) + optional timing =====
import os, glob, time, json, joblib, numpy as np, pandas as pd, tensorflow as tf
from PIL import Image
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ---------- Config / paths ----------
assert 'EFFICIENTNET_TFLITE_FP16' in globals() and os.path.exists(EFFICIENTNET_TFLITE_FP16), \
    "EFFICIENTNET_TFLITE_FP16 not found. Make sure Cell 4 for EfficientNet ran."
BACKBONE_TFLITE = EFFICIENTNET_TFLITE_FP16
TAG = "EfficientNetV2B0_fp16_FE"
SAVE_DIR = HYBRID_SAVE_DIR if 'HYBRID_SAVE_DIR' in globals() else "./hybrid_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# Expect features from Cell 4:
X_tr = globals().get('Xe_tr'); X_va = globals().get('Xe_va')
y_tr = globals().get('ye_tr'); y_va = globals().get('ye_va')
assert isinstance(X_tr, np.ndarray) and isinstance(X_va, np.ndarray), "Xe_tr/Xe_va missing."
assert isinstance(y_tr, np.ndarray) and isinstance(y_va, np.ndarray), "ye_tr/ye_va missing."
assert 'test_df' in globals() and len(test_df) > 0 and 'image_path' in test_df.columns, "test_df with image_path required."

X_train = np.vstack([X_tr, X_va])
y_train = np.concatenate([y_tr, y_va])
d_in = X_train.shape[1]            # should be 1280 for EfficientNetV2B0 pooled features
pca_n_default = 128 if d_in > 128 else None

# ---------- Recover heads (KNN/SVM_RBF/RF) from RAM or disk ----------
heads = {}
if 'heads' in globals() and isinstance(heads, dict) and heads:
    pass
else:
    if 'knn' in globals(): heads['KNN'] = knn
    if 'svm' in globals(): heads['SVM_RBF'] = svm
    if 'rf'  in globals(): heads['RandomForest'] = rf

def _try_load_head(path):
    try:
        return joblib.load(path)
    except Exception:
        return None

if not heads:
    cand = []
    cand += [os.path.join(SAVE_DIR, f"{TAG}_KNN.joblib"),
             os.path.join(SAVE_DIR, f"{TAG}_SVM_RBF.joblib"),
             os.path.join(SAVE_DIR, f"{TAG}_RandomForest.joblib")]
    cand += glob.glob(os.path.join(SAVE_DIR, "*_KNN.joblib"))
    cand += glob.glob(os.path.join(SAVE_DIR, "*_SVM_RBF.joblib"))
    cand += glob.glob(os.path.join(SAVE_DIR, "*_RandomForest.joblib"))
    for p in cand:
        est = _try_load_head(p)
        if est is None: continue
        if p.endswith("KNN.joblib"): heads.setdefault("KNN", est)
        elif p.endswith("SVM_RBF.joblib"): heads.setdefault("SVM_RBF", est)
        elif p.endswith("RandomForest.joblib"): heads.setdefault("RandomForest", est)

# If still missing, train simple versions now (uses same steps as earlier cell)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

def _mk_steps(use_pca=True, pca_n=pca_n_default):
    steps = [("scaler", StandardScaler())]
    if use_pca and pca_n:
        steps.append(("pca", PCA(n_components=pca_n, random_state=42)))
    return steps

if not heads:
    print("ℹ️ Training heads on EfficientNet features …")
    knn = Pipeline(_mk_steps(True) + [("knn", KNeighborsClassifier(n_neighbors=5, weights="distance", n_jobs=-1))]).fit(X_train, y_train)
    svm = Pipeline(_mk_steps(True) + [("svm", SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=42))]).fit(X_train, y_train)
    rf  = Pipeline(_mk_steps(True) + [("rf", RandomForestClassifier(
            n_estimators=600, max_depth=None, min_samples_leaf=1,
            class_weight="balanced_subsample", n_jobs=-1, random_state=42
         ))]).fit(X_train, y_train)
    heads = {"KNN": knn, "SVM_RBF": svm, "RandomForest": rf}
    joblib.dump(knn, os.path.join(SAVE_DIR, f"{TAG}_KNN.joblib"))
    joblib.dump(svm, os.path.join(SAVE_DIR, f"{TAG}_SVM_RBF.joblib"))
    joblib.dump(rf,  os.path.join(SAVE_DIR, f"{TAG}_RandomForest.joblib"))
    print("✅ Heads trained & saved.")

# ---------- Ensure each head matches current feature width; refit if needed ----------
def _needs_refit(pipe, xw):
    scaler = pipe.named_steps.get("scaler")
    pca    = pipe.named_steps.get("pca")
    if scaler is not None:
        mean = getattr(scaler, "mean_", None)
        if mean is not None and mean.shape[0] != xw:
            return True
    if pca is not None:
        comps = getattr(pca, "components_", None)
        if comps is not None and comps.shape[1] != xw:
            return True
    nfi = getattr(pipe, "n_features_in_", None)
    return (nfi is not None and nfi != xw)

def _refit_like(pipe, X, y):
    steps = []
    if "scaler" in pipe.named_steps:
        steps.append(("scaler", StandardScaler()))
    if "pca" in pipe.named_steps:
        old_pca = pipe.named_steps["pca"]
        n_comp = getattr(old_pca, "n_components", pca_n_default)
        if isinstance(n_comp, int): n_comp = min(n_comp, X.shape[1])
        steps.append(("pca", PCA(n_components=n_comp, random_state=42)))
    last_name, last_est = list(pipe.named_steps.items())[-1]
    new_pipe = Pipeline(steps + [(last_name, clone(last_est))]).fit(X, y)
    return new_pipe

for name in list(heads.keys()):
    if _needs_refit(heads[name], X_train.shape[1]):
        print(f"♻️ Re-fitting {name} on EfficientNet features (width {X_train.shape[1]}) …")
        heads[name] = _refit_like(heads[name], X_train, y_train)
        joblib.dump(heads[name], os.path.join(SAVE_DIR, f"{TAG}_{name}.joblib"))

# ---------- Distillation utils (to LR) & head TFLite export ----------
def _transform(pipe, X):
    Z = X
    if "scaler" in pipe.named_steps: Z = pipe.named_steps["scaler"].transform(Z)
    if "pca"    in pipe.named_steps: Z = pipe.named_steps["pca"].transform(Z)
    return Z

def fuse_head(pipe, X_train, y_train):
    Z_train = _transform(pipe, X_train)
    lr = LogisticRegression(max_iter=2000, random_state=42)
    try:
        prob = pipe.predict_proba(X_train)[:, 1]
        y_hard = (np.clip(prob, 1e-6, 1-1e-6) >= 0.5).astype(int)
        lr.fit(Z_train, y_hard)
    except Exception:
        lr.fit(Z_train, y_train)

    scaler = pipe.named_steps.get("scaler")
    pca    = pipe.named_steps.get("pca")
    scale = getattr(scaler, "scale_", None) if scaler is not None else None
    mean  = getattr(scaler, "mean_",  None) if scaler is not None else None
    P     = pca.components_ if pca is not None else None

    Wlr = lr.coef_.reshape(-1,1).astype(np.float32)
    blr = float(lr.intercept_.ravel()[0])

    Wp  = (P.T @ Wlr).astype(np.float32) if P is not None else Wlr
    Wf  = (Wp * (1.0 / scale.reshape(-1,1))).astype(np.float32) if scale is not None else Wp
    b_shift = - float((Wf.ravel() * mean).sum()) if mean is not None else 0.0
    bf = np.array([blr + b_shift], dtype=np.float32)
    return Wf, bf

def export_head_tflite(Wf, bf, out_path):
    D_in = Wf.shape[0]
    class HeadDenseSigmoid(tf.Module):
        def __init__(self, W, b):
            super().__init__()
            self.W = tf.Variable(W, trainable=False, dtype=tf.float32, name="W")
            self.b = tf.Variable(b, trainable=False, dtype=tf.float32, name="b")
        @tf.function(input_signature=[tf.TensorSpec([1, D_in], tf.float32, name="features")])
        def __call__(self, features):
            logits = tf.linalg.matmul(features, self.W) + self.b
            prob = tf.math.sigmoid(logits)
            return {"prob": prob}
    module = HeadDenseSigmoid(Wf, bf)
    concrete = module.__call__.get_concrete_function()
    conv = tf.lite.TFLiteConverter.from_concrete_functions([concrete], trackable_obj=module)
    tbytes = conv.convert()
    with open(out_path, "wb") as f: f.write(tbytes)
    print("💾 Head TFLite:", out_path, "|", round(os.path.getsize(out_path)/(1024*1024),3), "MB")
    return out_path

# ---------- Export all three heads ----------
head_paths = {}
for name, pipe in heads.items():
    Wf, bf = fuse_head(pipe, X_train, y_train)
    out_path = os.path.join(SAVE_DIR, f"{TAG}_HEAD_from_{name}_fused_fp32.tflite")
    head_paths[name] = export_head_tflite(Wf, bf, out_path)

# ---------- Optional: timing backbone + each head TFLite on 10 images ----------
def _prep_img(p, W, H, dtype):
    img = Image.open(p).convert("RGB").resize((W, H), Image.BILINEAR)
    x = np.asarray(img, dtype=np.float32)/255.0
    x = np.expand_dims(x, 0)
    if dtype == np.float16: x = x.astype(np.float16)
    elif dtype == np.float32: x = x.astype(np.float32)
    else: x = x.astype(dtype)
    return x

def _find_feat_idx(inter, d_in_guess):
    details = inter.get_tensor_details()
    sel = None
    for d in details:
        name = d["name"].decode() if isinstance(d["name"], bytes) else d["name"]
        shp = tuple(int(s) for s in (d.get("shape_signature", d["shape"])))
        if len(shp) in (1,2) and any(k in name.lower() for k in ("global_average_pool","global_max_pool",
                                                                 "avg_pool","mean","gap","pool")):
            sel = d
    if sel is None:
        for d in details:
            shp = tuple(int(s) for s in (d.get("shape_signature", d["shape"])))
            if len(shp) in (1,2) and shp[-1] == d_in_guess:
                sel = d; break
    return sel["index"] if sel is not None else inter.get_output_details()[0]["index"]

def time_two_tflites(backbone_path, head_path, img_paths, warmups=3, d_in_guess=d_in):
    b = tf.lite.Interpreter(model_path=backbone_path); b.allocate_tensors()
    bi = b.get_input_details()[0]; H,W = int(bi["shape"][1]), int(bi["shape"][2])
    feat_idx = _find_feat_idx(b, d_in_guess=d_in_guess)
    h = tf.lite.Interpreter(model_path=head_path); h.allocate_tensors()
    hi = h.get_input_details()[0]

    for _ in range(warmups):
        a = _prep_img(img_paths[0], W, H, bi["dtype"])
        b.set_tensor(bi["index"], a); b.invoke()
        f = b.get_tensor(feat_idx).astype(np.float32).reshape(1, d_in_guess)
        h.set_tensor(hi["index"], f); h.invoke()

    times=[]
    for p in img_paths:
        a = _prep_img(p, W, H, bi["dtype"])
        t0 = time.perf_counter()
        b.set_tensor(bi["index"], a); b.invoke()
        f = b.get_tensor(feat_idx).astype(np.float32).reshape(1, d_in_guess)
        h.set_tensor(hi["index"], f); h.invoke()
        _ = h.get_output_details()[0]
        t1 = time.perf_counter()
        times.append((t1-t0)*1000.0)

    t = np.array(times, np.float64)
    return dict(N=len(img_paths), avg_ms=round(t.mean(),3),
                std_ms=round(t.std(ddof=1),3) if len(t)>1 else 0.0,
                min_ms=round(t.min(),3), max_ms=round(t.max(),3))

sample_paths = list(test_df["image_path"])[:10] if len(test_df) >= 10 else list(test_df["image_path"])
timing_rows = []
for name, hpath in head_paths.items():
    metrics = time_two_tflites(BACKBONE_TFLITE, hpath, sample_paths, warmups=3, d_in_guess=d_in)
    timing_rows.append({"Head": name, "BackboneTFLite": BACKBONE_TFLITE, "HeadTFLite": hpath, **metrics})

df_timing = pd.DataFrame(timing_rows)
display(df_timing)
csv_out = os.path.join(SAVE_DIR, f"{TAG}_ALL_heads_latency.csv")
df_timing.to_csv(csv_out, index=False)

print("\n✅ Saved head files & timing CSV in:", SAVE_DIR)
for name, p in head_paths.items():
    print(f" - {name}: {p}")
print(" - timing:", csv_out)


💾 Head TFLite: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_from_KNN_fused_fp32.tflite | 0.006 MB
💾 Head TFLite: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_from_SVM_RBF_fused_fp32.tflite | 0.006 MB
💾 Head TFLite: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_from_RandomForest_fused_fp32.tflite | 0.006 MB


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


,Head,BackboneTFLite,HeadTFLite,N,avg_ms,std_ms,min_ms,max_ms
0,KNN,/content/drive/MyDrive/DiabeticProject/tflite/...,/content/drive/MyDrive/DiabeticProject/hybrid_...,10,235.305,106.849,88.400,366.775
1,SVM_RBF,/content/drive/MyDrive/DiabeticProject/tflite/...,/content/drive/MyDrive/DiabeticProject/hybrid_...,10,75.515,19.926,58.539,116.118
2,RandomForest,/content/drive/MyDrive/DiabeticProject/tflite/...,/content/drive/MyDrive/DiabeticProject/hybrid_...,10,86.424,26.184,54.409,126.373



✅ Saved head files & timing CSV in: /content/drive/MyDrive/DiabeticProject/hybrid_models
 - KNN: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_from_KNN_fused_fp32.tflite
 - SVM_RBF: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_from_SVM_RBF_fused_fp32.tflite
 - RandomForest: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_HEAD_from_RandomForest_fused_fp32.tflite
 - timing: /content/drive/MyDrive/DiabeticProject/hybrid_models/EfficientNetV2B0_fp16_FE_ALL_heads_latency.csv
